In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import random as rn

In [3]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show#, output_file #Descomentar para guardar el archivo .html
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [4]:
x1 = np.array([np.random.uniform(-20,20) for i in range(10000)])
x2 = np.array([np.random.uniform(-20,20) for i in range(10000)])

In [5]:
x1cuad = np.array([i**2 for i in x1])
x2cuad = np.array([i**2 for i in x2])

In [6]:
X = np.array([[x1[i],x2[i],x1cuad[i],x2cuad[i]] for i in range(len(x1))])

In [7]:
circulo1 = np.array([1 if np.sqrt((x1[i]+2)**2+(x2[i]+10)**2)<=2 else 0 for i in range(len(x1))])
circulo2 = np.array([1 if np.sqrt((x1[i]-1)**2+(x2[i]-3)**2)<=2 else 0 for i in range(len(x1))])
circulo3 = np.array([1 if np.sqrt((x1[i]-10)**2+(x2[i]+5)**2)<=2 else 0 for i in range(len(x1))])

In [8]:
circulos = circulo1+circulo2+circulo3

In [9]:
circulos = circulos.reshape(-1,1)

In [10]:
p = figure(width=800, plot_height=800, title=None)
p.circle(x1,x2,color=["red" if i==1 else "blue" for i in circulos])
show(p)

In [154]:
input_size=4
hidden_layer_size=9
output_layer_size=1
#Los placeholders son para los datos de entrada
x = tf.placeholder(tf.float32, shape=[len(x1),input_size ])
y_ = tf.placeholder(tf.float32, shape=[len(circulos),output_layer_size ]) #la y real
#La variables para lo que se va a ir calculando y modificando en el camino
W_layer1=tf.Variable(tf.random_uniform([input_size,hidden_layer_size], -1, 1), name="W_layer1")
W_layer2=tf.Variable(tf.random_uniform([hidden_layer_size,output_layer_size],-1, 1), name="W_layer2")
b_layer1 = tf.Variable(tf.zeros([hidden_layer_size]), name="b_layer1")
b_layer2 = tf.Variable(tf.zeros([output_layer_size]), name="b_layer2")

In [201]:
y = tf.sigmoid(tf.matmul(x,W_layer1)+b_layer1) #la de nuestra
hip = tf.sigmoid(tf.matmul(y, W_layer2)+b_layer2)
#lossfn = tf.reduce_mean(tf.reduce_sum((y_-y)**2)) #cuadratico
lossfn= tf.reduce_mean(( (y_ * tf.log(hip)) +  
  ((1 - y_) * tf.log(1.0 - hip)) ) * -1) # cross entropy
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(lossfn)

In [203]:
init = tf.initialize_all_variables()
sess = tf.Session()

In [209]:
sess.run(init)
for i in range(9000000):
    sess.run(train_step, feed_dict={x: X, y_: circulos})

In [210]:
correct_prediction = tf.equal(tf.round(hip),y_) # aca estams definiendo otras operacioes
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: X, y_: circulos})) # las corremos

0.9988


In [211]:
Y = sess.run(hip,feed_dict={x: X, y_: circulos})

In [11]:
[i for i in range(len(circulos)) if circulos[i]!=0];

In [214]:
Y = np.round(Y)

In [215]:
p = figure(width=800, plot_height=800, title=None)
p.circle(X[:,0],X[:,1],color=["red" if i==1 else "blue" for i in Y])
p.title.text = "Cículo predicho"
p.xaxis.axis_label = "X1"
p.yaxis.axis_label = "X2"
show(p)  

# SVM

In [95]:
from sklearn.svm import SVC

In [99]:
predic = SVC(C=100)

In [100]:
predic.fit(X,circulos)

/home/ollin/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [101]:
Y = predic.predict(X)
p = figure(width=800, plot_height=800, title=None)
p.circle(X[:,0],X[:,1],color=["red" if i==1 else "blue" for i in Y])
p.title.text = "Cículo predicho"
p.xaxis.axis_label = "X1"
p.yaxis.axis_label = "X2"
show(p)  

# k vecinos

In [12]:
from sklearn.neighbors import KNeighborsClassifier

In [13]:
neigh = KNeighborsClassifier(n_neighbors=3)

In [15]:
Y = neigh.fit(X, circulos)

/home/ollin/.pyenv/versions/anaconda2-4.1.1/lib/python2.7/site-packages/ipykernel/__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


In [17]:
Y = neigh.predict(X)

In [18]:
p = figure(width=800, plot_height=800, title=None)
p.circle(X[:,0],X[:,1],color=["red" if i==1 else "blue" for i in Y])
p.title.text = "Cículo predicho"
p.xaxis.axis_label = "X1"
p.yaxis.axis_label = "X2"
show(p)